In [1]:
import chess
import torch
import util
import pandas as pd
import pickle as pkl

In [2]:
bruv = pd.read_csv('games.csv')

In [3]:
games = pd.read_csv('games.csv')["moves"]

In [4]:
len(games)

20058

In [5]:
game_lengths = [len(game.split(" ")) for game in games]
# most common game length

most_common_gl = 53


In [6]:
name_to_games = dict()
for i, game in enumerate(games):
    moves = game.split(' ')
    if len(moves) == most_common_gl:
        if name_to_games.get(bruv["white_id"][i]) is None:
            name_to_games[bruv["white_id"][i]] = []
        name_to_games[bruv["white_id"][i]].append(moves)

In [7]:
len(list(name_to_games.keys()))

269

In [8]:
# count total games collected
total_games = 0
for name in name_to_games:
    total_games += len(name_to_games[name])
total_games

303

In [12]:
# memory usage go brrrr
unique_states = set()
for i, game in enumerate(games):
    board = chess.Board()
    for move in game.split():
        board.push_san(move)
        unique_states.add(util.board_to_list(board))
    print(i, i/len(games)*100, len(unique_states), end='\r')

KeyboardInterrupt: 

In [8]:
# save the unique states
with open('./data/lichess_unique_states.pkl', 'wb') as f:
    pkl.dump(unique_states, f)

In [9]:
len(unique_states)

1013462

In [3]:
unique_states = pkl.load(open('./data/lichess_unique_states.pkl', 'rb'))

In [4]:
states_tensor = torch.zeros(len(unique_states), 13, 8, 8)

In [5]:
states_tensor.shape

torch.Size([1013462, 13, 8, 8])

In [5]:
index = 0

In [6]:
# memory and time usage truly go brrr 
for state in unique_states:
    # state to tensor
    state_tensor = util.one_hot_board(state)
    states_tensor[index] = state_tensor
    index += 1
    print(index, index/len(unique_states)*100, end='\r')

In [9]:
# save 
torch.save(states_tensor, './data/lichess_unique_onehot_states.pt')